# ITREX - Leveraging Intel Optimizations for Enhanced Inference with Hugging Face

<img src="https://miro.medium.com/v2/resize:fit:1400/1*AfoKgyTN6l7xNg30GmbsLg.png" alt="Alt Text" style="width: 800px;"/>

Welcome to this developer-focused workshop, where we explore the integration of Intel extensions with Hugging Face models for optimized inference. The goal of this notebook is to demonstrate how developers can use Intel's extensions to achieve efficient and performant inference in production applications.

## Why Intel Optimizations Matter

In the realm of machine learning, particularly in NLP, the ability to perform efficient and speedy inference is crucial. By using the Intel extension for Transformers, we can load models directly from the Hugging Face Hub, like the "Intel/neural-chat-7b-v1-1" model, and optimize them for high-performance inference.

### Key Learning Points

- **Model Optimization**: Learn how to load and optimize Hugging Face models using Intel's neural compressor and extension APIs.
- **Streaming Output**: We'll use the TextStreamer functionality from Hugging Face Transformers to deliver a constant stream of tokens, enhancing the user experience by avoiding large text dumps.
- **Intel's Neural Chat Model**: Explore the "Intel/neural-chat-7b-v1-1" model, fine-tuned on Gaudi 2 processors, to understand its capabilities in generating text based on input prompts.
- **Practical Application**: Understand how these optimizations can be applied in real-world scenarios to deliver performant inference with minimal code.

By the end of this notebook, you'll have a practical understanding of how to apply Intel's optimizations to Hugging Face models for efficient inference.

Let's dive in and explore the power of optimized model inference!


In [1]:
!source /opt/intel/oneapi/setvars.sh #comment out if not running on Intel Developer Cloud Jupyter
!pip install transformers
!pip install intel_extension_for_transformers
!pip install tqdm
!pip install einops

 
:: WARNING: setvars.sh has already been run. Skipping re-execution.
   To force a re-execution of setvars.sh, use the '--force' option.
   Using '--force' can result in excessive use of your environment variables.
  
usage: source setvars.sh [--force] [--config=file] [--help] [...]
  --force        Force setvars.sh to re-run, doing so may overload environment.
  --config=file  Customize env vars using a setvars.sh configuration file.
  --help         Display this help message and exit.
  ...            Additional args are passed to individual env/vars.sh scripts
                 and should follow this script's arguments.
  
  Some POSIX shells do not accept command-line options. In that case, you can pass
  command-line options via the SETVARS_ARGS environment variable. For example:
  
  $ SETVARS_ARGS="ia32 --config=config.txt" ; export SETVARS_ARGS
  $ . path/to/setvars.sh
  
  The SETVARS_ARGS environment variable is cleared on exiting setvars.sh.
  
Defaulting to user installatio

#### Importing Required Libraries

This cell sets the foundation for our model optimization and text generation tasks. We import:
- `AutoTokenizer` and `TextStreamer` from Hugging Face's `transformers` library, crucial for tokenizing our input text and streaming the model's output.
- `AutoModelForCausalLM` from `intel_extension_for_transformers`, which is a specialized version of the model class optimized for Intel hardware.

In [2]:
from transformers import AutoTokenizer, TextStreamer
from intel_extension_for_transformers.transformers import AutoModelForCausalLM

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


#### Model and Prompt Setup

Here, we specify the model and the initial text prompt for our text generation task.
- `model_name`: We set this to "Intel/neural-chat-7b-v1-1", a model fine-tuned on Intel's hardware, available on the Hugging Face model hub.
- `prompt`: This is our starting text for the model to generate from, setting the context for the text generation.


In [3]:
model_name = "Intel/neural-chat-7b-v1-1"     # Hugging Face model_id or local model
prompt = "Once upon a time, there existed a fisherman at sea,"

#### Tokenizer Initialization and Input Preparation

In this cell, we initialize the tokenizer with our chosen model and prepare our input text for the model.
- `tokenizer`: Loaded with the `AutoTokenizer.from_pretrained` method, tailored for our specific model.
- `inputs`: The prompt is tokenized to be fed into the model.
- `streamer`: An instance of `TextStreamer` is created with our tokenizer, enabling efficient and user-friendly text generation output.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
streamer = TextStreamer(tokenizer)

#### Model Loading and Text Generation

This is where the action happens:
- We load our model using `AutoModelForCausalLM.from_pretrained`, with `load_in_4bit=True` to enable optimized inference.
- The model's `generate` function is called with the `streamer` parameter, which enables streaming output of the text. We set `max_new_tokens` to 300 to control the length of the generated text.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)
outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300)

2024-01-30 06:22:11 [INFO] CPU device is used.
2024-01-30 06:22:11 [INFO] Applying Weight Only Quantization.
2024-01-30 06:22:11 [INFO] Using LLM runtime.


cmd: ['python', PosixPath('/home/uad6b15e0ae3d5e407195ab5f044a50f/.local/lib/python3.9/site-packages/intel_extension_for_transformers/llm/runtime/graph/scripts/convert_mpt.py'), '--outfile', 'runtime_outs/ne_mpt_f32.bin', '--outtype', 'f32', 'Intel/neural-chat-7b-v1-1']


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.
transformer.wte.weight torch.Size([50279, 4096]) torch.float32
transformer.blocks.0.norm_1.weight torch.Size([4096]) torch.float32
transformer.blocks.0.attn.Wqkv.weight torch.Size([12288, 4096]) torch.float32
transformer.blocks.0.attn.out_proj.weight torch.Size([4096, 4096]) torch.float32
transformer.blocks.0.norm_2.weight torch.Size([4096]) torch.float32
transformer.blocks.0.ffn.up_proj.weight torch.Size([16384, 4096]) torch.float32
transformer.blocks.0.ffn.down_proj.weight torch.Size([4096, 16384]) torch.float32
transformer.blocks.1.norm_1.weight torch.Size([4096]) torch.float32
transformer.blocks.1.attn.Wqkv.weight torch.Size([12288, 4096]) torch.float32
transformer.blocks.1.attn.out_proj.weight torch.Size([4096, 4096]) torch.float32
transformer.blocks.1.norm_2.weight torch.Size([4096]) torch.float32
transformer.blocks.1.ffn.up_proj.weight torch.Size([

In [ ]:
model.generate(inputs, streamer=streamer, max_new_tokens=300)

# Conclusion and Discussion

### Conclusion

This workshop demonstrated the practical application of Intel optimizations in conjunction with Hugging Face's powerful Transformers library. We explored the nuances of model loading, tokenization, and efficient text generation using Intel's neural compressor and extension APIs.

### Discussion

The skills and knowledge gained here are essential for developers looking to implement optimized NLP models in production environments. The ability to generate text in a streamed manner and leverage Intel's hardware optimizations showcases the potential for building responsive and efficient AI-powered applications.

As we continue to advance in the field of AI, understanding and applying such optimizations will be crucial for developers to stay ahead in creating high-performance, scalable, and user-friendly applications.
